In [ ]:
import pandas
import itertools
import csv

#creating a list of chromosomes to iterate through
chroms = ['chr%s' % i for i in range(1, 23) + ['X', 'Y', 'M']]

#create a list of pass subcategories
pass_subcats = ['PASS[0/1]', 'PASS[0]', 'PASS[.]', 'PASS[1]', 'PASS[1/1]', 'PASS[1/2]', 'PASS[0/0]']

#create a list of columns for the dataframe matrix. This represents all of the possible rcf filter
#categories
rcf_cats = ['not present', 'nc'] + pass_subcats

#create a list of indexes for the dataframe matrix. This represents all of the possible gvcf
#filter categories
gvcf_cats = ['not present']
with open('/40TB_3/InovaGenomes/Filter_combinations') as filters:
    filter_reader = csv.reader(filters, delimiter = '\t')
    for row in filter_reader:
        gvcf_cats.append(row[0])
gvcf_cats = gvcf_cats + pass_subcats
gvcf_cats.remove('PASS')

In [ ]:
#This code assumes that the input gvcf pos-->cat file is one line longer than rcf pos-->cat file, 
#because the first line of the gvcf pos--> cat file displays a count of pass[00] in the chromosome. 
#Assumes that the second line of the gvcf inputstream alligns with the first line of the rcf input stream.
#Does not account for mismatching lines. If there are two consecutive lines at the same position,
#will add that to the count. 

# when there is a deletion in the GVCF file, there will be at least 1 line and probably 2 lines
# in the RCF file.

for chrom in chroms[0:1]: #switch to full list later
    matrix = pandas.DataFrame(0, index= gvcf_cats, columns=rcf_cats)
    with open('%s position and category.gvcf'%chrom) as gvcf, open('%s position and category.rcf'%chrom) as rcf:
        gvcf_reader = csv.reader(gvcf, delimiter = '\t')
        rcf_reader = csv.reader(rcf, delimiter = '\t')
        pass_00_row = next(gvcf_reader)
        pass_00_count = pass_00_row[1]
        matrix['not present'].loc['PASS[0/0]'] = pass_00_count
        count = 0
        row_rcf = next(rcf_reader)
        for row_gvcf in gvcf_reader:
            while True:
                rcf_pos = int(row_rcf[0])+1 # convert base 0 to base 1 to match gvcf
                gvcf_pos = int(row_gvcf[0])
                gvcf_cat = row_gvcf[1]
                rcf_cat = row_rcf[1]
                if (gvcf_pos == rcf_pos):
                    matrix.set_value(gvcf_cat, rcf_cat, matrix[rcf_cat].loc[gvcf_cat] + 1)                
                    try:
                        row_rcf = next(rcf_reader)
                    except:
                        break
                if gvcf_pos < rcf_pos:
                    break
        matrix.to_csv(path_or_buf ='category matrix for %s.csv'%chrom, sep = '\t', header = True, index = True)